In [1]:
import numpy as np
np.random.seed(42)
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

from keras.models import Model ,Sequential
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate, SpatialDropout2D, Dropout, Flatten
from keras.layers import CuDNNGRU, Bidirectional, GlobalAveragePooling1D, GlobalMaxPooling1D
from keras.preprocessing import text, sequence
from keras.callbacks import Callback
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import KFold
from keras import regularizers
import warnings

warnings.filterwarnings('ignore')



EMBEDDING_FILE = '../data/crawl-300d-2M.vec'

train = pd.read_csv('../data/train.csv.zip')
test = pd.read_csv('../data/test.csv.zip')
submission = pd.read_csv('../data/sample_submission.csv.zip')

X_train = train["comment_text"].fillna("fillna").values
y_train = train[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values
X_test = test["comment_text"].fillna("fillna").values

/media/stas/fastdata/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras import regularizers

In [3]:
from keras.models import Model ,Sequential

In [4]:
from keras.layers import Input, Dense, Embedding, SpatialDropout1D, concatenate, Reshape, SpatialDropout2D, Dropout, Flatten

In [5]:
max_features = 100000
maxlen = 650
embed_size = 300

tokenizer = text.Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(X_train) + list(X_test))
X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)
x_train = sequence.pad_sequences(X_train, maxlen=maxlen)
x_test = sequence.pad_sequences(X_test, maxlen=maxlen)


def get_coefs(word, *arr): return word, np.asarray(arr, dtype='float32')
embeddings_index = dict(get_coefs(*o.rstrip().rsplit(' ')) for o in open(EMBEDDING_FILE))

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [6]:
class RocAucEvaluation(Callback):
    def __init__(self, validation_data=(), interval=1):
        super(Callback, self).__init__()

        self.interval = interval
        self.X_val, self.y_val = validation_data

    def on_epoch_end(self, epoch, logs={}):
        if epoch % self.interval == 0:
            y_pred = self.model.predict(self.X_val, verbose=0)
            score = roc_auc_score(self.y_val, y_pred)
            print("\n ROC-AUC - epoch: %d - score: %.6f \n" % (epoch+1, score))


def get_model(units=80):
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = SpatialDropout1D(0.2)(x)
    x = Bidirectional(CuDNNGRU(units, return_sequences=True))(x)
    avg_pool = GlobalAveragePooling1D()(x)
    max_pool = GlobalMaxPooling1D()(x)
    conc = concatenate([avg_pool, max_pool])
    outp = Dense(6, activation="sigmoid")(conc)
    
    model = Model(inputs=inp, outputs=outp)
    ad = Adam(lr=0.005)
    model.compile(loss='binary_crossentropy',
                  optimizer= ad,
                  metrics=['accuracy'])

    return model
#model = get_model()

In [7]:
def get_pool(unots=80, reg=0.001):
    inp = Input(shape=(maxlen, ))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
    x = Bidirectional(CuDNNGRU(unots, return_sequences=True))(x)
    
    x = SpatialDropout1D(0.5)(x)
    
    x = Bidirectional(CuDNNGRU(unots, return_sequences=True,return_state=True))(x)    
    avg_pool = GlobalAveragePooling1D()(x[0])
    max_pool = GlobalMaxPooling1D()(x[0])
    conc = concatenate([avg_pool, max_pool,x[1],x[2]])
    #outp = Dropout(0.5)(conc)
    outp = Dense(6, activation="sigmoid")(conc)

    model = Model(inputs=inp, outputs=outp)
    ad = Adam(lr=0.005)
    model.compile(loss='binary_crossentropy',
                      optimizer= ad,
                      metrics=['accuracy'])
    return model

In [8]:
def get(units = 80):
    model = Sequential()
    model.add(Embedding(max_features, embed_size, weights=[embedding_matrix]))
    model.add(Bidirectional(CuDNNGRU(units, return_sequences=True)))
    model.add(SpatialDropout1D(0.5))
    model.add(Bidirectional(CuDNNGRU(units)))
    #model.add(Dropout(0.5))
    model.add(Dense(6, activation="sigmoid"))
    ad = Adam(lr=0.005)
    model.compile(loss='binary_crossentropy',
                      optimizer= ad,
                      metrics=['accuracy'])
    return model

In [15]:
batch_size = 500
epochs = 5
model = get_pool()
X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.90, random_state=233)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),
                 callbacks=[RocAuc], verbose=1)
#model.save('../models/vanilla_Polled_GRU_onlypooling/2epoch.h5')

#y_pred = model.predict(x_test, batch_size=1024)
#submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred
#submission.to_csv('../submits/vanilla_Polled_GRU_onlypooling/2epoch.csv', index=False)

Train on 143613 samples, validate on 15958 samples
Epoch 1/5
143613/143613 [==============================] - 137s 951us/step - loss: 0.0672 - acc: 0.9757 - val_loss: 0.0447 - val_acc: 0.9830

 ROC-AUC - epoch: 1 - score: 0.982955 

Epoch 2/5
143613/143613 [==============================] - 134s 935us/step - loss: 0.0382 - acc: 0.9851 - val_loss: 0.0433 - val_acc: 0.9834

 ROC-AUC - epoch: 2 - score: 0.986208 

Epoch 3/5
143613/143613 [==============================] - 135s 937us/step - loss: 0.0303 - acc: 0.9878 - val_loss: 0.0459 - val_acc: 0.9827

 ROC-AUC - epoch: 3 - score: 0.984967 

Epoch 4/5
  5000/143613 [>.............................] - ETA: 2:06 - loss: 0.0239 - acc: 0.9907

KeyboardInterrupt: 

In [8]:
batch_size = 300
epochs = 5
model = get_pool()
X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.90, random_state=233)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),
                 callbacks=[RocAuc], verbose=1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/5
143613/143613 [==============================] - 155s 1ms/step - loss: 0.0584 - acc: 0.9785 - val_loss: 0.0446 - val_acc: 0.9835

 ROC-AUC - epoch: 1 - score: 0.983991 

Epoch 2/5
143613/143613 [==============================] - 155s 1ms/step - loss: 0.0381 - acc: 0.9850 - val_loss: 0.0473 - val_acc: 0.9822

 ROC-AUC - epoch: 2 - score: 0.986527 

Epoch 3/5
143613/143613 [==============================] - 155s 1ms/step - loss: 0.0310 - acc: 0.9875 - val_loss: 0.0487 - val_acc: 0.9823

 ROC-AUC - epoch: 3 - score: 0.985882 

Epoch 4/5
143613/143613 [==============================] - 156s 1ms/step - loss: 0.0262 - acc: 0.9897 - val_loss: 0.0524 - val_acc: 0.9815

 ROC-AUC - epoch: 4 - score: 0.985036 

Epoch 5/5
 12600/143613 [=>............................] - ETA: 2:17 - loss: 0.0233 - acc: 0.9910

KeyboardInterrupt: 

In [ ]:
batch_size = 300
epochs = 5
model = get_pool(60)
X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.90, random_state=233)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),
                 callbacks=[RocAuc], verbose=1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/5
143613/143613 [==============================] - 132s 922us/step - loss: 0.0561 - acc: 0.9796 - val_loss: 0.0442 - val_acc: 0.9832

 ROC-AUC - epoch: 1 - score: 0.987218 

Epoch 2/5
129000/143613 [=========================>....] - ETA: 12s - loss: 0.0366 - acc: 0.9854

In [ ]:
batch_size = 500
epochs = 5
model = get_pool(40)
X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.90, random_state=233)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),
                 callbacks=[RocAuc], verbose=1)

In [ ]:
batch_size = 500
epochs = 5
model = get_pool(25)
X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.90, random_state=233)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),
                 callbacks=[RocAuc], verbose=1)

In [17]:
batch_size = 400
epochs = 5
model = get_pool(40)
X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.90, random_state=233)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),
                 callbacks=[RocAuc], verbose=1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/5
143613/143613 [==============================] - 93s 644us/step - loss: 0.0552 - acc: 0.9801 - val_loss: 0.0417 - val_acc: 0.9840

 ROC-AUC - epoch: 1 - score: 0.988462 

Epoch 2/5
143613/143613 [==============================] - 90s 625us/step - loss: 0.0333 - acc: 0.9867 - val_loss: 0.0461 - val_acc: 0.9824

 ROC-AUC - epoch: 2 - score: 0.987222 

Epoch 3/5
 40400/143613 [=======>......................] - ETA: 1:04 - loss: 0.0223 - acc: 0.9911

KeyboardInterrupt: 

In [21]:
batch_size = 400
epochs = 5
model = get_pool(25)
X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.90, random_state=233)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),
                 callbacks=[RocAuc], verbose=1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/5
143613/143613 [==============================] - 91s 631us/step - loss: 0.1334 - acc: 0.9652 - val_loss: 0.1089 - val_acc: 0.9714

 ROC-AUC - epoch: 1 - score: 0.983103 

Epoch 2/5
143613/143613 [==============================] - 90s 629us/step - loss: 0.0991 - acc: 0.9659 - val_loss: 0.1065 - val_acc: 0.9627

 ROC-AUC - epoch: 2 - score: 0.985528 

Epoch 3/5
143613/143613 [==============================] - 90s 626us/step - loss: 0.0931 - acc: 0.9661 - val_loss: 0.1088 - val_acc: 0.9628

 ROC-AUC - epoch: 3 - score: 0.983477 

Epoch 4/5
 64400/143613 [============>.................] - ETA: 48s - loss: 0.0864 - acc: 0.9675

KeyboardInterrupt: 

In [ ]:
batch_size = 300
epochs = 5
model = get_pool(25)
X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.90, random_state=233)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),
                 callbacks=[RocAuc], verbose=1)

In [14]:
batch_size = 400
epochs = 5
#model = get(100)

X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.90, random_state=233)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),
                 callbacks=[RocAuc], verbose=1)

Train on 143613 samples, validate on 15958 samples
Epoch 1/5
143613/143613 [==============================] - 90s 624us/step - loss: 0.0547 - acc: 0.9809 - val_loss: 0.0429 - val_acc: 0.9834

 ROC-AUC - epoch: 1 - score: 0.989048 

Epoch 2/5
143613/143613 [==============================] - 90s 624us/step - loss: 0.0336 - acc: 0.9865 - val_loss: 0.0435 - val_acc: 0.9828

 ROC-AUC - epoch: 2 - score: 0.988309 

Epoch 3/5
143613/143613 [==============================] - 90s 624us/step - loss: 0.0241 - acc: 0.9904 - val_loss: 0.0507 - val_acc: 0.9811

 ROC-AUC - epoch: 3 - score: 0.986570 

Epoch 4/5
 21200/143613 [===>..........................] - ETA: 1:14 - loss: 0.0145 - acc: 0.9947

KeyboardInterrupt: 

In [ ]:
batch_size = 500
epochs = 5
model = get(125)

X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.90, random_state=233)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),
                 callbacks=[RocAuc], verbose=1)

In [6]:
gru_for_stack.head()

,id,identity_hate,insult,obscene,severe_toxic,threat,toxic
0,0000997932d777bf,0.000027,0.000102,0.000097,0.000015,0.000037,0.000405
1,000103f0d9cfb60f,0.000062,0.000082,0.000109,0.000019,0.000015,0.000741
2,000113f07ec002fd,0.000060,0.000136,0.000473,0.000076,0.000090,0.001095
3,0001b41b1c6bb37e,0.000018,0.000059,0.000094,0.000022,0.000018,0.000213
4,0001d958c54c6e35,0.000400,0.004207,0.001973,0.000241,0.000584,0.034808


In [9]:
0.01model.load_weights("../models/vanilla_gru_cv_10/fold_1.best.hdf5")
y_pred = model.predict(x_test, batch_size=1024)

In [10]:
%%time
for i in range(2,11):
    model.load_weights(f"../models/vanilla_gru_cv_10/fold_{i}.best.hdf5")
    y_pred *= model.predict(x_test, batch_size=1024)


CPU times: user 2min, sys: 20.7 s, total: 2min 20s
Wall time: 2min 15s


In [11]:
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred ** 0.1
submission.to_csv('../submits/vanilla_gru_cv_10/submitiongeomaverage.csv', index=False)

In [ ]:
batch_size = 32
epochs = 1

X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.95, random_state=233)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),
                 callbacks=[RocAuc], verbose=1)
model.save('../models/vanilla_Polled_GRU_onlypooling/2epoch.h5')

y_pred = model.predict(x_test, batch_size=1024)
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred
submission.to_csv('../submits/vanilla_Polled_GRU_onlypooling/2epoch.csv', index=False)

In [8]:
batch_size = 32
epochs = 1

X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.95, random_state=233)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),
                 callbacks=[RocAuc], verbose=1)
model.save('../models/vanilla_Polled_GRU_onlypooling/2epoch.h5')

y_pred = model.predict(x_test, batch_size=1024)
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred
submission.to_csv('../submits/vanilla_Polled_GRU_onlypooling/2epoch.csv', index=False)

Train on 151592 samples, validate on 7979 samples
Epoch 1/1
151592/151592 [==============================] - 234s 2ms/step - loss: 0.0353 - acc: 0.9860 - val_loss: 0.0416 - val_acc: 0.9842

 ROC-AUC - epoch: 1 - score: 0.989550 



In [9]:
batch_size = 32
epochs = 1

X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.95, random_state=233)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),
                 callbacks=[RocAuc], verbose=1)
model.save('../models/vanilla_Polled_GRU_onlypooling/3epoch.h5')

y_pred = model.predict(x_test, batch_size=1024)
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred
submission.to_csv('../submits/vanilla_Polled_GRU_onlypooling/3epoch.csv', index=False)

Train on 151592 samples, validate on 7979 samples
Epoch 1/1
151592/151592 [==============================] - 236s 2ms/step - loss: 0.0291 - acc: 0.9884 - val_loss: 0.0448 - val_acc: 0.9835

 ROC-AUC - epoch: 1 - score: 0.988765 



In [10]:
batch_size = 32
epochs = 1

X_tra, X_val, y_tra, y_val = train_test_split(x_train, y_train, train_size=0.95, random_state=233)
RocAuc = RocAucEvaluation(validation_data=(X_val, y_val), interval=1)

hist = model.fit(X_tra, y_tra, batch_size=batch_size, epochs=epochs, validation_data=(X_val, y_val),
                 callbacks=[RocAuc], verbose=1)
model.save('../models/vanilla_Polled_GRU_onlypooling/4epoch.h5')

y_pred = model.predict(x_test, batch_size=1024)
submission[["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]] = y_pred
submission.to_csv('../submits/vanilla_Polled_GRU_onlypooling/4epoch.csv', index=False)

Train on 151592 samples, validate on 7979 samples
Epoch 1/1
151592/151592 [==============================] - 236s 2ms/step - loss: 0.0229 - acc: 0.9910 - val_loss: 0.0493 - val_acc: 0.9827

 ROC-AUC - epoch: 1 - score: 0.988463 

